# Lista 07 - Comparando Classificadores

# Exercício 01:

Analise o desempenho do kNN e de uma Regressão Logística Regularizada para **pelo menos um** dos conjuntos de dados abaixo:

* [Avaliação de carros](http://archive.ics.uci.edu/ml/datasets/Car+Evaluation)
* [Avaliação de vinhos](http://archive.ics.uci.edu/ml/datasets/Wine+Quality)
* [Resultados de partidas do jogo Dota](http://archive.ics.uci.edu/ml/datasets/Dota2+Games+Results) (desafiador!)

Para a questão, faça as seguintes tarefas:

* Realize treino, validação e teste
* Compare as métricas no teste
* Reporte a precisão, revocação, F1 e a matriz de confusão

Como já estamos no fim da matéria, você pode agora fazer uso da biblioteca scikit-learn. Afinal, no dia a dia, não implementamos tudo do zero. Abaixo temos os imports que vocês precisam. Leiam a API da biblioteca para saber como fazer uso da mesma.

In [1]:
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from sklearn.neighbors import KNeighborsClassifier

# Um fator importante é que o SKLearn não cria conjuntos de validação para você. Você tem algumas abordagens,
# uma é realizar um novo split no treino. Outra é fazer uso de classificadores com CV no fim.
# Tipo LogisticRegressionCV (ver na API). Por fim, você pode fazer uso da classe GridSearchCV.
# Leia a documentação da mesma.
from sklearn.model_selection import train_test_split

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.model_selection import  GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

%matplotlib inline

plt.rcParams.update({'font.size': 15})
np.random.seed(2016058581)

# Base de dados

In [3]:
# Dataset reading
df = pd.read_csv("winequality-red.csv", ';')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Separação dos conjuntos de treino (80%) e teste (20%)

In [4]:
X = df.iloc[:, :-1].copy()
Y = df['quality'].copy()
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.2)

---
# KNN

In [5]:
knn = KNeighborsClassifier()
knn = GridSearchCV(knn, {'n_neighbors':range(1,51)}, cv=5, iid=False)
knn.fit(xTrain, yTrain)
print('Melhor k (entre 1 e 50) encontrado para o KNN: %d'%(knn.best_params_)['n_neighbors'])

Melhor k (entre 1 e 50) encontrado para o KNN: 1


## Métricas

In [6]:
yPred = knn.predict(xTest)
prf1 = np.array(precision_recall_fscore_support(yTest, yPred)[:-1])
metrics = pd.DataFrame(prf1, columns=range(3,9), index=['Precision', 'Recall', 'F1'])
metrics

,3,4,5,6,7,8
Precision,0.333333,0.100000,0.677966,0.572464,0.479167,0.0
Recall,0.200000,0.083333,0.592593,0.622047,0.638889,0.0
F1,0.250000,0.090909,0.632411,0.596226,0.547619,0.0


## Matriz de confusão

In [7]:
cm = confusion_matrix(yTest, yPred, labels=range(3,9))
pd.DataFrame(cm, index=range(3,9), columns=range(3,9))

,3,4,5,6,7,8
3,1,1,3,0,0,0
4,1,1,2,6,2,0
5,1,5,80,41,7,1
6,0,2,31,79,13,2
7,0,1,2,10,23,0
8,0,0,0,2,3,0


---
# Regressão Logística

In [8]:
lr = LogisticRegressionCV(cv=5, max_iter=10000, multi_class='auto')
lr.fit(xTrain, yTrain)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=10000,
           multi_class='auto', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

## Métricas

In [9]:
yPred2 = lr.predict(xTest)
prf12 = np.array(precision_recall_fscore_support(yTest, yPred2)[:-1])
metrics2 = pd.DataFrame(prf12, columns=range(3,9), index=['Precision', 'Recall', 'F1'])
metrics2

/home/ramon/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,3,4,5,6,7,8
Precision,0.0,0.0,0.668790,0.543478,0.440000,0.0
Recall,0.0,0.0,0.777778,0.590551,0.305556,0.0
F1,0.0,0.0,0.719178,0.566038,0.360656,0.0


## Matriz de confusão

In [10]:
cm = confusion_matrix(yTest, yPred2, labels=range(3,9))
pd.DataFrame(cm, index=range(3,9), columns=range(3,9))

,3,4,5,6,7,8
3,0,0,4,1,0,0
4,0,0,6,6,0,0
5,0,0,105,30,0,0
6,0,0,39,75,13,0
7,0,0,3,22,11,0
8,0,0,0,4,1,0


# Comparação

In [11]:
accKNN = accuracy_score(yTest, yPred)
accLR = accuracy_score(yTest, yPred2)

print('Acurácia do KNN: %.2f%%'%(accKNN*100))
print('Acurácia da Regressão Logística: %.2f%%'%(accLR*100))

Acurácia do KNN: 57.50%
Acurácia da Regressão Logística: 59.69%


Explique e discuta sobre os resultados encontrados no campo abaixo.

A acurácia alcançada pela Regressão Logística é superior à acurácia alcançada pelo KNN, mas uma diferença não muito significativa (cerca de 2%). Ambos os algoritmos conseguiram uma revocação e precisão boas para vinhos com qualidade mediana (5), porém, o resultado foi ruim para vinhos com qualidade alta (7 e 8).